In [293]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import talib
import numpy as np
import pandas as pd


In [294]:
def plotCandlesticks(data):
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.3,
        row_heights=[0.7, 0.7],  # Adjust row heights as needed
        subplot_titles=('Candlestick Chart', 'MACD')
    )
    
    # plot all candlesticks in data
    candlestick = go.Candlestick(x=data.index, 
                             open=data['Open'],  
                             high=data['High'], 
                             low=data['Low'], 
                             close=data['Close'])
    fig.add_trace(candlestick, row=1, col=1)

    fig.add_trace(go.Scatter(x=data.index, y=data['sma_50'], mode='lines', name='sma_100', line=dict(color='orange', width=1)), row=1, col=1)

    # add MACD info below first graph
    fig.add_trace(go.Scatter(x=data.index, y=data['macd_line'], mode='lines', name='macd', line=dict(color='black', width=1)), row=2, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['macd_signal_line'], mode='lines', name='macd_signal', line=dict(color='blue', width=1)), row=2, col=1)
    fig.add_trace(go.Bar(x=data.index, y=data['macd_histogram'], name='macd_histogram', marker_color='red'), row=2, col=1)

    # make lists of data points with buy or sell signals
    golden = data[ data['golden_death'] == 'BUY' ].index
    death = data[ data['golden_death'] == 'SELL' ].index

    buy_data = data[ data['rsi'] < 40 ]
    buy = data[ 
        # strong signals
        (data['stick_sand_indicator'] == 'BUY') |
        (data['breakaway_indicator'] == 'BUY') |
        (data['takuri_indicator'] == 'BUY') |
        (data['kicking_indicator'] == 'BUY') |
        (data['3methods_indicator'] == 'BUY') |
        (data['abandoned_baby_indicator'] == 'BUY') |

        (data['inverted_hammer_indicator'] == 'BUY') |
        (data['counterattack_indicator'] == 'BUY') |

        # weak signals
        (data['marubozu_indicator'] == 'BUY') |
        (data['morning_star_indicator'] == 'BUY') |


        # needs rsi check
        ( (data['macd_indicator'] == 'BUY') & (data['rsi'] < 40) ) |
        ( (data['engulfing_indicator'] == 'BUY') & (data['rsi'] < 40) )
    ].index

    sell_data = data[ data['rsi'] > 60 ]
    sell = data[ 
        # strong signals
        (data['stick_sand_indicator'] == 'SELL') |
        (data['breakaway_indicator'] == 'SELL') |
        (data['takuri_indicator'] == 'SELL') |
        (data['dragonfly_doji_indicator'] == 'SELL') |
        (data['kicking_indicator'] == 'SELL') |
        (data['3methods_indicator'] == 'SELL') |
        (data['abandoned_baby_indicator'] == 'SELL') |

        (data['counterattack_indicator'] == 'SELL') |

        # weak signals
        (data['marubozu_indicator'] == 'SELL') |
        (data['evening_star_indicator'] == 'SELL') |
        (data['hanging_man_indicator'] == 'SELL') |

        # needs rsi check
        ( (data['macd_indicator'] == 'SELL') & (data['rsi'] > 60) ) |
        ( (data['engulfing_indicator'] == 'SELL') & (data['rsi'] > 60) )
    ].index

    # plot buy and sell signals as vertical dashed lines
    for date in golden:
        fig.add_vline(x=date, line_width=2, line_dash="dash", line_color="yellow", row=1, col=1)
    for date in death:
        fig.add_vline(x=date, line_width=2, line_dash="dash", line_color="black", row=1, col=1)
    for date in buy:
        fig.add_vline(x=date, line_width=2, line_dash="dash", line_color="green", row=1, col=1)
    for date in sell:
        fig.add_vline(x=date, line_width=2, line_dash="dash", line_color="red", row=1, col=1)

    fig.update_layout(  
        height=1200, 
        width=800, 
        title_text='Stock Analysis'
    )
    fig.show()

    return

In [295]:
def addCandlePatterns(data):
    # add engulfing and star patterns to data
    data['engulfing'] = talib.CDLENGULFING(data['Open'], data['High'], data['Low'], data['Close'])
    data['morning_star'] = talib.CDLMORNINGSTAR(data['Open'], data['High'], data['Low'], data['Close'])
    data['evening_star'] = talib.CDLEVENINGSTAR(data['Open'], data['High'], data['Low'], data['Close'])

    data['engulfing_indicator'] = 'HOLD'
    data.loc[ (data['engulfing'] == 100), 'engulfing_indicator'] = 'BUY'
    data.loc[ (data['engulfing'] == -100), 'engulfing_indicator'] = 'SELL'

    data['morning_star_indicator'] = 'HOLD'
    data.loc[ (data['morning_star'] == 100), 'morning_star_indicator'] = 'BUY'

    data['evening_star_indicator'] = 'HOLD'
    data.loc[ (data['evening_star'] == -100), 'evening_star_indicator'] = 'SELL'

    # find stcik sandwhich pattern
    data['stick_sandwich'] = talib.CDLSTICKSANDWICH(data['Open'], data['High'], data['Low'], data['Close'])
    data['stick_sand_indicator'] = 'HOLD'
    data.loc[ (data['stick_sandwich'] != 0) & (data['Open'] > data['Close']), 'stick_sand_indicator'] = 'BUY'
    data.loc[ (data['stick_sandwich'] != 0) & (data['Open'] < data['Close']), 'stick_sand_indicator'] = 'SELL'

    # find identical 3 crows
    data['iden_3_crows'] = talib.CDLIDENTICAL3CROWS(data['Open'], data['High'], data['Low'], data['Close'])
    data['iden_3_crows_indicator'] = 'HOLD'
    data.loc[ data['iden_3_crows'] != 0, 'iden_3_crows_indicator'] = 'SELL'

    # find breakaways
    data['breakaway'] = talib.CDLBREAKAWAY(data['Open'], data['High'], data['Low'], data['Close'])
    data['breakaway_indicator'] = 'HOLD'
    data.loc[ data['breakaway'] == -100, 'breakaway_indicator'] = 'SELL'
    data.loc[ data['breakaway'] == 100, 'breakaway_indicator'] = 'BUY'

    # find kicking pattern
    data['kicking'] = talib.CDLKICKING(data['Open'], data['High'], data['Low'], data['Close'])
    data['kicking_indicator'] = 'HOLD'
    data.loc[ data['kicking'] == -100, 'kicking_indicator'] = 'SELL'
    data.loc[ data['kicking'] == 100, 'kicking_indicator'] = 'BUY'

    # find takuri candles
    data['takuri'] = talib.CDLTAKURI(data['Open'], data['High'], data['Low'], data['Close'])
    data['takuri_indicator'] = 'HOLD'
    data.loc[ data['takuri'] != 0, 'takuri_indicator'] = 'BUY'

    # find marubozu candles
    data['marubozu'] = talib.CDLMARUBOZU(data['Open'], data['High'], data['Low'], data['Close'])
    data['marubozu_indicator'] = 'HOLD'
    data.loc[ data['marubozu'] == 100, 'marubozu_indicator'] = 'BUY'
    data.loc[ data['marubozu'] == -100, 'marubozu_indicator'] = 'SELL'

    # find dragonfly doji
    data['dragonfly_doji'] = talib.CDLDRAGONFLYDOJI(data['Open'], data['High'], data['Low'], data['Close'])
    data['dragonfly_doji_indicator'] = 'HOLD'
    data.loc[ data['dragonfly_doji'] != 0, 'dragonfly_doji_indicator'] = 'SELL'

    # find 3 methods pattern
    data['3methods'] = talib.CDLRISEFALL3METHODS(data['Open'], data['High'], data['Low'], data['Close'])
    data['3methods_indicator'] = 'HOLD'
    data.loc[ data['3methods'] == 100, '3methods_indicator'] = 'BUY'
    data.loc[ data['3methods'] == -100, '3methods_indicator'] = 'SELL'

    data['abandoned_baby'] = talib.CDLABANDONEDBABY(data['Open'], data['High'], data['Low'], data['Close'])
    data['abandoned_baby_indicator'] = 'HOLD'
    data.loc[ data['abandoned_baby'] == 100, 'abandoned_baby_indicator'] = 'BUY'
    data.loc[ data['abandoned_baby'] == -100, 'abandoned_baby_indicator'] = 'SELL'

    data['hanging_man'] = talib.CDLHANGINGMAN(data['Open'], data['High'], data['Low'], data['Close'])
    data['hanging_man_indicator'] = 'HOLD'
    data.loc[ data['hanging_man'] != 0, 'hanging_man_indicator'] = 'SELL'

    data['inverted_hammer'] = talib.CDLINVERTEDHAMMER(data['Open'], data['High'], data['Low'], data['Close'])
    data['inverted_hammer_indicator'] = 'HOLD'
    data.loc[ data['inverted_hammer'] != 0, 'inverted_hammer_indicator'] = 'BUY'

    data['counterattack'] = talib.CDLCOUNTERATTACK(data['Open'], data['High'], data['Low'], data['Close'])
    data['counterattack_indicator'] = 'HOLD'
    data.loc[ data['counterattack'] == 100, 'counterattack_indicator'] = 'BUY'
    data.loc[ data['counterattack'] == -100, 'counterattack_indicator'] = 'SELL'

    # drop unnecessary columns
    data = data.drop(columns=['engulfing', 'morning_star', 'evening_star', 'stick_sandwich', 'breakaway', 'kicking', 'takuri', 'marubozu', 
                              'dragonfly_doji', '3methods', 'abandoned_baby', 'hanging_man', 'inverted_hammer', 'counterattack'])

    # print(data[ data['counterattack'] != 0])

    return data

In [296]:
def addMovAvg(data):
    # add moving averages to data
    data['sma_20'] = talib.SMA(data['Close'], 20)
    data['sma_50'] = talib.SMA(data['Close'], 50)
    data['sma_100'] = talib.SMA(data['Close'], 100)
    data['sma_200'] = talib.SMA(data['Close'], 200)

    data['ema_8'] = talib.EMA(data['Close'], 8)
    data['ema_10'] = talib.EMA(data['Close'], 10)
    data['ema_13'] = talib.EMA(data['Close'], 13)
    data['ema_21'] = talib.EMA(data['Close'], 21)
    data['ema_50'] = talib.EMA(data['Close'], 50)

    # drop rows with missing values
    data = data.dropna()

    # set conditions for buy or sell signals for moving averages
    conditions = [
        ( (data['Close'] < data['sma_20']) & (data['sma_20'] < data['sma_50']) & (data['sma_50'] < data['sma_200']) ),
        ( (data['Close'] > data['sma_20']) & (data['sma_20'] > data['sma_50']) & (data['sma_50'] > data['sma_200']) )
    ]
    choices = ['BUY', 'SELL']

    # add buy or sell signals to data
    data['sma_comp_indicator'] = np.select(conditions, choices, default='HOLD')

    return data

In [297]:
def addCrossoverInd(data):
    # add golden cross or death cross signal to data
    data['sma_long_diff'] = data['sma_100'] - data['sma_200']
    data['golden_death'] = 'HOLD'
    data.loc[(data['sma_long_diff'].shift(1) < 0) & (data['sma_long_diff'] > 0), 'golden_death'] = 'BUY'
    data.loc[(data['sma_long_diff'].shift(1) > 0) & (data['sma_long_diff'] < 0), 'golden_death'] = 'SELL'

    # add long and short term triple EMA signals to data
    data['ema_diff_1'] = data['ema_21'] - data['ema_50']
    data['long_ema_cross_indicator'] = 'HOLD'
    data.loc[(data['ema_diff_1'].shift(1) < 0) & (data['ema_diff_1'] > 0) & (data['ema_10'] > data['ema_21']), 'long_ema_cross_indicator'] = 'BUY'
    data.loc[(data['ema_diff_1'].shift(1) > 0) & (data['ema_diff_1'] < 0) & (data['ema_10'] < data['ema_21']), 'long_ema_cross_indicator'] = 'SELL'

    data['ema_diff_2'] = data['ema_13'] - data['ema_21']
    data['short_ema_cross_indicator'] = 'HOLD'
    data.loc[(data['ema_diff_2'].shift(1) < 0) & (data['ema_diff_2'] > 0) & (data['ema_8'] > data['ema_13']), 'short_ema_cross_indicator'] = 'BUY'
    data.loc[(data['ema_diff_2'].shift(1) > 0) & (data['ema_diff_2'] < 0) & (data['ema_8'] < data['ema_13']), 'short_ema_cross_indicator'] = 'SELL'

    # drop unnecessary columns
    data = data.drop(columns=['sma_long_diff', 'ema_diff_1', 'ema_diff_2'])
    
    return data

In [298]:
def addRSI_ATR(data):
    RSI_PERIOD = 14
    data['rsi'] = talib.RSI(data['Close'], RSI_PERIOD)

    data['atr'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)

    return data

In [299]:
def addMACD(data):    
    macd, signal, hist = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

    data['macd_line'] = macd
    data['macd_signal_line'] = signal
    data['macd_histogram'] = hist

    data['macd_diff'] = data['macd_line'] - data['macd_signal_line']
    data['macd_indicator'] = 'HOLD'
    data.loc[(data['macd_diff'].shift(1) < 0) & (data['macd_diff'] > 0), 'macd_indicator'] = 'BUY'
    data.loc[(data['macd_diff'].shift(1) > 0) & (data['macd_diff'] < 0), 'macd_indicator'] = 'SELL'

    return data


In [300]:
def backtest():    
# main
    import os


    startDate = '2010-12-31'
    endDate = '2024-12-31'

    stocks = ['AAPL', 'GOOG', 'TSLA', 'AMZN', 'MTN', 'MSFT', 'NVDA', 'F', 'AMD']
    profit_margin = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
    stop_loss = [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 1000]

    res_stop_loss = []
    res_total_profit = []
    res_profit_per_trade = []
    res_ror = []

    # loop through all possible combinations of profit margin and stop loss
    # for prof in profit_margin:
    for stop in stop_loss:
        #initialize empty csv with headers
        temp_df = pd.DataFrame({'date': [], 'stock_purchased': [], 'price': [], 'profit': [], 'num_shares': [], 'stop_val': [], 'total_invested': []}) 
        temp_df.to_csv(f'./data/{stop}.csv', index=False)

        for stock in stocks:
            # get all necessary data for the stock
            data = yf.download(stock, start=startDate)
            data = addCandlePatterns(data)
            data = addMovAvg(data)
            # data = addCrossoverInd(data)
            data = addRSI_ATR(data)
            data = addMACD(data)

            # find all indicies with buy signals
            buy = data[ 
                # strong signals
                (data['stick_sand_indicator'] == 'BUY') |
                (data['breakaway_indicator'] == 'BUY') |
                (data['takuri_indicator'] == 'BUY') |
                (data['kicking_indicator'] == 'BUY') |
                (data['3methods_indicator'] == 'BUY') |
                (data['abandoned_baby_indicator'] == 'BUY') |

                (data['inverted_hammer_indicator'] == 'BUY') |
                (data['counterattack_indicator'] == 'BUY') |

                # weak signals
                (data['marubozu_indicator'] == 'BUY') |
                (data['morning_star_indicator'] == 'BUY') |


                # needs rsi check
                ( (data['macd_indicator'] == 'BUY') & (data['rsi'] < 40) ) |
                ( (data['engulfing_indicator'] == 'BUY') & (data['rsi'] < 40) )
            ].index

            # find all indicies with sell signals
            sell = data[ 
                # strong signals
                (data['stick_sand_indicator'] == 'SELL') |
                (data['breakaway_indicator'] == 'SELL') |
                (data['takuri_indicator'] == 'SELL') |
                (data['dragonfly_doji_indicator'] == 'SELL') |
                (data['kicking_indicator'] == 'SELL') |
                (data['3methods_indicator'] == 'SELL') |
                (data['abandoned_baby_indicator'] == 'SELL') |

                (data['counterattack_indicator'] == 'SELL') |

                # weak signals
                (data['marubozu_indicator'] == 'SELL') |
                (data['evening_star_indicator'] == 'SELL') |
                (data['hanging_man_indicator'] == 'SELL') |

                # needs rsi check
                ( (data['macd_indicator'] == 'SELL') & (data['rsi'] > 60) ) |
                ( (data['engulfing_indicator'] == 'SELL') & (data['rsi'] > 60) )
            ].index

            # initialize variables
            date = []
            stock_purchased = []
            price = []
            num_shares = []
            profit = []
            stop_val = []
            total_invested = []
            tot_money_used = 0
            curr_stop = 0
            curr_shares = 0

            # for every index in each stock
            # if that day has a buy signal, buy the stock and update variables
            # if that day drops below the stop loss, sell the stock and update variables
            for index in data.index:
                curr_price = data.loc[index, 'Close']
                if index in buy:
                    date.append(index)
                    stock_purchased.append(stock)
                    price.append(curr_price)
                    curr_shares += 1/curr_price
                    num_shares.append(curr_shares)
                    profit.append(-1)
                    tot_money_used += 1
                    total_invested.append(tot_money_used)
                    stop_val.append(stop)
                    curr_stop = curr_price - (stop * data.loc[index, 'atr'])
                elif curr_price < curr_stop:
                    date.append(index)
                    stock_purchased.append(stock)
                    price.append(curr_price)
                    num_shares.append(0)
                    profit.append(curr_shares * curr_price)
                    stop_val.append(stop)
                    total_invested.append(tot_money_used)
                    curr_shares = 0
                    curr_stop = 0   
            if curr_shares != 0:
                end_price = data.iloc[-1]['Close']
                date.append("END")
                stock_purchased.append(stock)
                price.append(end_price)
                num_shares.append(0)
                profit.append(curr_shares * end_price)
                stop_val.append(stop)
                total_invested.append(tot_money_used)

            df = pd.DataFrame({'date': date, 'stock_purchased': stock_purchased, 'price': price, 'profit': profit, 'num_shares': num_shares, 'stop_val': stop_val, 'total_invested': total_invested}) 
            df.to_csv(f'./data/{stop}.csv', mode='a', header=False, index=False)

    directory = '.\data'
    files = os.listdir(directory)
    csv_files = [f for f in files if f.endswith('.csv')]

    for file in csv_files:
        df = pd.read_csv(f'./data/{file}')
        res_stop_loss.append(df['stop_val'].iloc[0])
        res_total_profit.append(sum(df['profit']))
        res_profit_per_trade.append(sum(df['profit']) / len(df['profit']))
        res_ror.append(sum(df['profit']) / sum(df['total_invested']))

    print(pd.DataFrame({
        'stop loss': res_stop_loss,
        'total profit': res_total_profit,
        'profit per trade': res_profit_per_trade,
        'ror': res_ror
    }))
    print("Max Profits: ", max(res_total_profit))


In [301]:
def import_data(stock, startDate):
    data = yf.download(stock, start=startDate, interval='15m')
    data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=7)
    data.reset_index(inplace=True)
    return data

In [302]:
from backtesting import Strategy
from backtesting import Backtest
import seaborn as sns
import matplotlib.pyplot as plt

stock = 'GOOG'
startDate = '2024-06-10'
data = import_data(stock, startDate)
addCandlePatterns(data)
addMovAvg(data)
addCrossoverInd(data)
addMACD(data)
addRSI_ATR(data)
buy = data[ 
        # strong signals
        (data['stick_sand_indicator'] == 'BUY') |
        (data['breakaway_indicator'] == 'BUY') |
        (data['takuri_indicator'] == 'BUY') |
        (data['kicking_indicator'] == 'BUY') |
        (data['3methods_indicator'] == 'BUY') |
        (data['abandoned_baby_indicator'] == 'BUY') |
        (data['golden_death'] == 'BUY') |
        (data['inverted_hammer_indicator'] == 'BUY') |
        (data['counterattack_indicator'] == 'BUY') |

        # weak signals
        (data['marubozu_indicator'] == 'BUY') |
        (data['morning_star_indicator'] == 'BUY') |


        # needs rsi check
        ( (data['macd_indicator'] == 'BUY') & (data['rsi'] < 40) ) |
        ( (data['engulfing_indicator'] == 'BUY') & (data['rsi'] < 40) )
    ].index
sell = data[ 
        # strong signals
        (data['stick_sand_indicator'] == 'SELL') |
        (data['breakaway_indicator'] == 'SELL') |
        (data['takuri_indicator'] == 'SELL') |
        (data['dragonfly_doji_indicator'] == 'SELL') |
        (data['kicking_indicator'] == 'SELL') |
        (data['3methods_indicator'] == 'SELL') |
        (data['abandoned_baby_indicator'] == 'SELL') |
        (data['counterattack_indicator'] == 'SELL') |
        (data['golden_death'] == 'SELL') |

        # weak signals
        (data['marubozu_indicator'] == 'SELL') |
        (data['evening_star_indicator'] == 'SELL') |
        (data['hanging_man_indicator'] == 'SELL') |

        # needs rsi check
        ( (data['macd_indicator'] == 'SELL') & (data['rsi'] > 60) ) |
        ( (data['engulfing_indicator'] == 'SELL') & (data['rsi'] > 60) )
    ].index
data['FinalSignal'] = 0
for index in buy:
    data['FinalSignal'][index] = 1
for index in sell:
    data['FinalSignal'][index] = -1

half = len(data) // 2
data = data[half:]
# print(data)

def SIGNAL():
        return data['FinalSignal']

class MyStrategy(Strategy):
    stop_loss_coef = 1.5
    take_profit_coef = 2.2

    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
    
    def next(self):
        super().next()
        stop_loss_value = self.stop_loss_coef * self.data.ATR[-1]
        price = 0.5
        
        if (self.signal1 == 1) and (len(self.trades) == 0):
            sl1 = min(self.data.Close[-1], self.data.Close[-1] - stop_loss_value)
            tp1 = max(self.data.Close[-1], self.data.Close[-1] + (self.take_profit_coef*stop_loss_value) )

            self.buy( sl=sl1, tp=tp1, size=price )

        elif (self.signal1 == -1) and (len(self.trades) == 0):
            sl1 = max(self.data.Close[-1], self.data.Close[-1] + stop_loss_value)
            tp1 = min(self.data.Close[-1], self.data.Close[-1] - (self.take_profit_coef*stop_loss_value) )
            
            self.sell( sl=sl1, tp=tp1, size=price )

bt = Backtest(data, MyStrategy, cash=1000, margin = 1, commission=.002)
results = bt.run()
print(results)
print(results._trades)
plotCandlesticks(data)

# stats, heatmap = bt.optimize(stop_loss_coef=[i/10 for i in range(11, 26)],
#                     take_profit_coef=[i/10 for i in range(11, 26)],
#                     maximize='Return [%]', max_tries=300,
#                         random_state=0,
#                         return_heatmap=True)
# print(stats)
# print(stats['_strategy'])
# print(stats['_trades'])

# heatmap_df = heatmap.unstack()
# plt.figure(figsize=(14, 10))
# sns.heatmap(heatmap_df, annot=True, cmap='viridis', fmt='.0f')
# plt.show()
# plotCandlesticks(data)

[*********************100%%**********************]  1 of 1 completed
C:\Users\18053\AppData\Local\Temp\ipykernel_10436\3226389860.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\18053\AppData\Local\Temp\ipykernel_10436\498177951.py:58: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_index

Start                                   267.0
End                                     533.0
Duration                                266.0
Exposure Time [%]                   70.786517
Equity Final [$]                   979.291548
Equity Peak [$]                   1000.383235
Return [%]                          -2.070845
Buy & Hold Return [%]                4.842955
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.131475
Avg. Drawdown [%]                   -1.077732
Max. Drawdown Duration                  258.0
Avg. Drawdown Duration                  130.0
# Trades                                 12.0
Win Rate [%]                        16.666667
Best Trade [%]                       0.909439
Worst Trade [%]                     -0.976723
Avg. Trade [%]                    